## Introduction

This project uses the Foursquare API to explore neighborhoods in the City of Toronto. The most common venue categories in each neighborhood are obtained, and then k-means cluster analysis is done according to these grouped features in the neighborhoods. Finally, the Folium library is used to visualize the neighborhoods in the City of Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Fetch and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in the City of Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

download all the dependencies that are needed.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## 1.  Scrape the Postal Data of the City of Toronto in the Wikipedia Page

Get the table here https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

Import the library to open URLs

In [2]:
import urllib.request

Specify the URL of the Wikipedia page

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Query the page and put the HTML data into a variable

In [4]:
page = urllib.request.urlopen(url)

Import the functions from Beautiful Soup

In [5]:
from bs4 import BeautifulSoup

Specify a parser format and store it in a new variable called ‘soup’ 

In [6]:
soup = BeautifulSoup(page, "lxml")

Find the table in the corresponding wikipedia page

In [7]:
all_tables=soup.find_all("table")

In [8]:
right_table=soup.find('table', class_='wikitable sortable')

Loop through the rows of the table

In [9]:
postal_codes=[]
boroughs=[]
neighborhoods=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_codes.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhoods.append(cells[2].find(text=True))

In [10]:
postal_codes = [postal_code.rstrip('\n') for postal_code in postal_codes] 
boroughs = [borough.rstrip('\n') for borough in boroughs] 
neighborhoods = [neighborhood.rstrip('\n') for neighborhood in neighborhoods] 

#### Tranform and Clean the Data into a *Pandas* Dataframe

In [11]:
import pandas as pd

In [12]:
Toronto_FSAs = pd.DataFrame(postal_codes,columns=['Postal Code'])
Toronto_FSAs['Borough'] = boroughs
Toronto_FSAs['Neighborhood'] = neighborhoods

Drop the rows of postal code which is not assigned to a borough

In [13]:
Toronto_FSAs = Toronto_FSAs[Toronto_FSAs.Borough != 'Not assigned'].reset_index(drop=True)

Take a look at the data

In [14]:
Toronto_FSAs.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now, incorporate each neighborhood with its specific coordinate

Read the csv file that contains corresponding the latitude and longitude information 

In [15]:
coordinate = pd.read_csv("http://cocl.us/Geospatial_data")

Add the coordinate information and name the dataframe 'neighborhood' for later processing

In [16]:
neighborhoods = Toronto_FSAs.merge(coordinate, on = 'Postal Code')

Take a look at the data

In [17]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Check the number of boroughs and neighborhoods in the dataframe.

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [19]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [20]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'U5V2GMVCLOJB5YISLWF2NI5OEATSPCGV2OPJMPTZCSAEO0LA' 
CLIENT_SECRET = 'S0WCQXWVVHE0L2QMBVX20QPMNZZXQ2MNOK5XJNNHKDDW2KX0' 
VERSION = '20200101' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: U5V2GMVCLOJB5YISLWF2NI5OEATSPCGV2OPJMPTZCSAEO0LA
CLIENT_SECRET:S0WCQXWVVHE0L2QMBVX20QPMNZZXQ2MNOK5XJNNHKDDW2KX0


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [22]:
neighborhoods.loc[0, 'Neighborhood']

'Parkwoods'

Get the neighborhood's latitude and longitude values.

In [23]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now, get the top 100 venues that are in Parkwoods within a radius of 500 meters.

First, create the GET request URL. Name the URL **url**.

In [24]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?client_id=U5V2GMVCLOJB5YISLWF2NI5OEATSPCGV2OPJMPTZCSAEO0LA&client_secret=S0WCQXWVVHE0L2QMBVX20QPMNZZXQ2MNOK5XJNNHKDDW2KX0&v=20200101&ll=43.7532586,-79.3296565&radius=500&limit=100'

Send the GET request and examine the resutls

In [25]:
results = requests.get(url).json()
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 245,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

#### define the function **get_category_type** that extracts the category of the venue from the Foursquare lab.

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns, remove '.' for all column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/opos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


And how many venues were returned by Foursquare?

In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Create a function to repeat the same process to all the neighborhoods

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [30]:
# type your answer here

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Check the size of the resulting dataframe

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(6935, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
4,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café


Check how many venues were returned for each neighborhood

In [32]:
count_venues = toronto_venues.groupby('Neighborhood').size().to_frame('Number of Venues')

In [33]:
count_venues.reset_index()

,Neighborhood,Number of Venues
0,Agincourt,61
1,"Alderwood, Long Branch",45
2,"Bathurst Manor, Wilson Heights, Downsview North",40
3,Bayview Village,17
4,"Bedford Park, Lawrence Manor East",75
5,Berczy Park,100
6,"Birch Cliff, Cliffside West",14
7,"Brockton, Parkdale Village, Exhibition Place",100
8,"Business reply mail Processing Centre, South C...",100
9,"CN Tower, King and Spadina, Railway Lands, Har...",68


#### Find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 348 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture 

examine the new dataframe size.

In [36]:
toronto_onehot.shape

(6935, 348)

#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture 

#### Confirm the new size

In [38]:
toronto_grouped.shape

(99, 348)

#### Print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index() #transpose
    temp.columns = ['venue','freq'] #rename column name
    temp = temp.iloc[1:] #get rid of 1st row, containing 'neighborhood'
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.18
1           Coffee Shop  0.05
2         Shopping Mall  0.05
3  Cantonese Restaurant  0.03
4           Pizza Place  0.03


----Alderwood, Long Branch----
              venue  freq
0              Park  0.07
1              Café  0.04
2       Pizza Place  0.04
3      Burger Joint  0.04
4  Toy / Game Store  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Park  0.12
1  Coffee Shop  0.08
2  Gas Station  0.05
3         Bank  0.05
4  Pizza Place  0.05


----Bayview Village----
         venue  freq
0         Park  0.12
1  Gas Station  0.12
2        Trail  0.12
3         Bank  0.12
4         Café  0.06


----Bedford Park, Lawrence Manor East----
                venue  freq
0              Bakery  0.09
1  Italian Restaurant  0.08
2         Coffee Shop  0.08
3    Sushi Restaurant  0.07
4          Bagel Shop  0.05


----Berczy Park----
                venue  freq
0         Coffe

#### Put them into a *pandas* dataframe

First, define a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Coffee Shop,Pizza Place,Breakfast Spot,Gym / Fitness Center,Food Court,Cantonese Restaurant,Japanese Restaurant,Bakery
1,"Alderwood, Long Branch",Park,Bank,Café,Clothing Store,Coffee Shop,Toy / Game Store,Grocery Store,Burger Joint,Pizza Place,Gas Station
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Coffee Shop,Bank,Pizza Place,Gas Station,Mediterranean Restaurant,French Restaurant,Bus Line,Baseball Field,Sandwich Place
3,Bayview Village,Park,Gas Station,Trail,Bank,Japanese Restaurant,Chinese Restaurant,Grocery Store,Skating Rink,Café,Intersection
4,"Bedford Park, Lawrence Manor East",Bakery,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bagel Shop,Pub,Café,Sandwich Place,Fast Food Restaurant,Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [42]:
#### set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([2, 1, 0, 0, 1, 4, 0, 4, 1, 4, 1, 4, 1, 4, 4, 4, 2, 0, 4, 1, 1, 2,
       1, 2, 2, 4, 4, 2, 1, 4, 1, 4, 2, 2, 2, 4, 4, 2, 0, 1], dtype=int32)

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [44]:
toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True)

toronto_merged.head(7)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Pharmacy,Coffee Shop,Park,Intersection,Gas Station,Bank,Supermarket,Fast Food Restaurant,Fish & Chips Shop,Café
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Fast Food Restaurant,Gym,Middle Eastern Restaurant,Shoe Store,Grocery Store,Bank,Italian Restaurant,Spa,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Restaurant,Park,Café,Diner,Japanese Restaurant,Pub,Italian Restaurant,Farmers Market,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Dessert Shop,Furniture / Home Store,Sandwich Place,Vietnamese Restaurant,Toy / Game Store,Latin American Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Café,Japanese Restaurant,Park,Ramen Restaurant,Restaurant,Supermarket,Comic Shop,Beer Bar,Gastropub
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,0,Pharmacy,Bank,Shopping Mall,Bus Stop,Bakery,Golf Course,Grocery Store,Liquor Store,Café,Skating Rink
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Zoo Exhibit,Fast Food Restaurant,Restaurant,Pizza Place,Chinese Restaurant,Women's Store,Bank,Caribbean Restaurant,Liquor Store,Other Great Outdoors


Finally, visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [46]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,0,Pharmacy,Bank,Shopping Mall,Bus Stop,Bakery,Golf Course,Grocery Store,Liquor Store,Café,Skating Rink
11,Etobicoke,0,Park,Convenience Store,Sandwich Place,Fish & Chips Shop,Bank,Bakery,Theater,Electronics Store,Restaurant,Café
12,Scarborough,0,Park,Hotel,Italian Restaurant,Burger Joint,Grocery Store,Gym,Gym / Fitness Center,Breakfast Spot,Design Studio,Filipino Restaurant
28,North York,0,Park,Coffee Shop,Bank,Pizza Place,Gas Station,Mediterranean Restaurant,French Restaurant,Bus Line,Baseball Field,Sandwich Place
39,North York,0,Park,Gas Station,Trail,Bank,Japanese Restaurant,Chinese Restaurant,Grocery Store,Skating Rink,Café,Intersection
50,North York,0,Park,Electronics Store,Bakery,Asian Restaurant,Vietnamese Restaurant,Coffee Shop,Bank,Shopping Mall,Japanese Restaurant,Italian Restaurant
51,Scarborough,0,Park,Harbor / Marina,Coffee Shop,Pizza Place,Beach,Grocery Store,Pharmacy,Fast Food Restaurant,Ice Cream Shop,Pub
58,Scarborough,0,Park,Restaurant,Filipino Restaurant,General Entertainment,Golf Course,Gym,Café,Gym Pool,Diner,Thai Restaurant
88,Etobicoke,0,Park,Indian Restaurant,Grocery Store,Café,Coffee Shop,Bakery,Pharmacy,Soccer Field,Bar,General Entertainment


#### Cluster 2

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Dessert Shop,Furniture / Home Store,Sandwich Place,Vietnamese Restaurant,Toy / Game Store,Latin American Restaurant
7,North York,1,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Bank,Sandwich Place,Gym,Middle Eastern Restaurant,Italian Restaurant,Park
13,North York,1,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Bank,Sandwich Place,Gym,Middle Eastern Restaurant,Italian Restaurant,Park
16,York,1,Italian Restaurant,Coffee Shop,Bank,Café,Pizza Place,Grocery Store,Trail,Sandwich Place,Caribbean Restaurant,Restaurant
19,East Toronto,1,Coffee Shop,Pub,Grocery Store,Breakfast Spot,Japanese Restaurant,Bakery,Sandwich Place,BBQ Joint,Ice Cream Shop,Beach
21,York,1,Coffee Shop,Italian Restaurant,Pizza Place,Furniture / Home Store,Bakery,Sushi Restaurant,Japanese Restaurant,Grocery Store,Mexican Restaurant,Café
23,East York,1,Indian Restaurant,Bakery,Coffee Shop,Grocery Store,Restaurant,Sandwich Place,Gym,Supermarket,Burger Joint,Sushi Restaurant
26,Scarborough,1,Coffee Shop,Clothing Store,Bakery,Indian Restaurant,Sandwich Place,Gas Station,Restaurant,Pharmacy,Sporting Goods Shop,Bank
29,East York,1,Coffee Shop,Restaurant,Sandwich Place,Bank,Furniture / Home Store,Burger Joint,Electronics Store,Indian Restaurant,Grocery Store,Pizza Place
33,North York,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Park,Chinese Restaurant,Bank,Gas Station,Sandwich Place,Japanese Restaurant,Restaurant


#### Cluster 3

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,3,Donut Shop,Farm,National Park,Zoo,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


#### Cluster 4

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Restaurant,Park,Café,Diner,Japanese Restaurant,Pub,Italian Restaurant,Farmers Market,Performing Arts Venue
4,Downtown Toronto,4,Coffee Shop,Café,Japanese Restaurant,Park,Ramen Restaurant,Restaurant,Supermarket,Comic Shop,Beer Bar,Gastropub
9,Downtown Toronto,4,Coffee Shop,Gastropub,Café,Japanese Restaurant,Restaurant,Italian Restaurant,Theater,Vegetarian / Vegan Restaurant,Plaza,Burrito Place
15,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Hotel,Plaza,Gastropub,Concert Hall,Italian Restaurant,American Restaurant,Liquor Store
20,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,Park,Plaza,Concert Hall,Italian Restaurant,American Restaurant,Farmers Market
24,Downtown Toronto,4,Coffee Shop,Park,Café,Restaurant,Burger Joint,Pizza Place,Theater,Italian Restaurant,Japanese Restaurant,Supermarket
25,Downtown Toronto,4,Café,Coffee Shop,Indian Restaurant,Italian Restaurant,Mexican Restaurant,Bar,Park,Korean Restaurant,Grocery Store,Ice Cream Shop
30,Downtown Toronto,4,Coffee Shop,Park,Café,Plaza,Japanese Restaurant,Restaurant,Yoga Studio,Sandwich Place,Supermarket,Baseball Stadium
31,West Toronto,4,Café,Italian Restaurant,Bar,Coffee Shop,Bakery,Park,Sushi Restaurant,Restaurant,Thai Restaurant,Brazilian Restaurant
35,East York,4,Café,Greek Restaurant,Coffee Shop,Pizza Place,Park,Bakery,Ice Cream Shop,Ethiopian Restaurant,Cosmetics Shop,Pub


#### Cluster 5

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).